<a href="https://colab.research.google.com/github/yokabicarpmaz/ME462_ControlSystemsTools/blob/master/transfer_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TRANSFER FUNCTION 

Let's start with adding necessary libraries !


Do NOT forget to run codes in order !

In [0]:
try :
  !pip install control
  !pip install cowsay
  import control as cnt
  import matplotlib.pyplot as plt
  import numpy as np
  import sympy as sy
  import math
  import cowsay
  from itertools import combinations
  cowsay.tux("All libraries have been successfully downloaded!")
except : 
   print("\033[1m"+"WARNING!!!LIBRARIES COULD NOT BE DOWNLOADED. PLEASE TRY AGAIN !"+"\033[0m")

Transfer functions specify the relationship between input and output.

$G(s) = \frac{b_ms^m+b_{m-1}s^{m-1}+...+b_{1}s+b_0}{a_ns^n+a_{n-1}s^{n-1}+...+a_{1}s+a_0} =\frac{N(s)}{D(s)}$

To create transfer function in laplace form :

Numerator and denominator coefficients should be given in a matrix form in descending order then cnt.tf(num,den) or cnt.TransferFunction(num,den) commands are used.




In [0]:
num = [1,2]
den = [1,5,6]
TF = cnt.TransferFunction(num,den) 
print("G(s)", TF, sep = "=")

If denominator's order is higher than numerator: ($n\geq m$)

 the roots of D(s) = 0 which is also named as characteristic equation give us the poles of the system. Poles can be determined by cnt.pole() command.

 the roots of N(s) = 0 give us zeros of the system. Poles can be determined by cnt.zero() command.
 

In [0]:
Poles = cnt.pole(TF)
Zeros = cnt.zero(TF)
print("Poles of the system=", Poles)
print("Zeros of the system=", Zeros)

Zeros and poles can also create the transfer function. You can follow the next code block and try to understand the task or you can use directly tf_from_pz(poles = [  ], zeros = [  ]) command to get the transfer function.

In [0]:
def get_coefficients(ps):  #the function which takes input from tf_from_fz function and returns the denominator&numerator coefficients 
    n = len(ps)            
    ps = [-p for p in ps]    
    coefficients = []        
    for i, p in enumerate(ps):   
        comb = list(combinations(ps, n-i))
        products = [np.prod(c) for c in comb]
        coefficient = sum(products)
        if abs(coefficient.imag) > 1e-5:
            print("Imaginary coefficients are not supported by control toolbox.")
            return
        else:
            coefficients.append(coefficient.real)
    coefficients.append(1)
    return coefficients[::-1]        

def tf_from_pz(poles = [], zeros = []):   #the function which takes poles and zeros and returns transfer function
    den = get_coefficients(poles) 
    num = get_coefficients(zeros)         
    return TransferFunction(num, den)


In [0]:
TF = tf_from_pz(poles = [1+1j,1-1j, 3, 5, 7], zeros = [8,9,123])

print(f"Poles are {TF.pole()}")
print(f"Zeros are {TF.zero()}")
print("TF =", TF)

Important Notes up to here:
  - Transfer function is a property of a system and is independent of input.
  - Transfer functions of different physical systems can be the same. 



---



---


Example 1 - First Order System


---



---


<figure>
<center>
<img src='https://i0.wp.com/programmerworld.co/wp-content/uploads/2019/04/sprinmassdampersystem.png?w=715&ssl=1' />
<figcaption>Figure 1</figcaption></center>

A mass(m)-spring(k)-damper(b) system is shown in *Figure 1*. The system input is Force F and output is mass displacement x.


*   Obtain the input-output relationship as $m\ddot{x}+b\dot{x}+kx=F$ .
*   In order to get transfer function, we should get the laplace transform of input-output expression.
$ms^2X(s)+bsX(s)+kX(s)=F(s)$
then 
$\dfrac {output} {input} = \dfrac {X(s)} {F(s)} = \dfrac {1} {ms^2+bs+k}  $.




**Displacement of mass can be described as force multiplied by the transfer function. We can see the transfer function effects on displacement.** 

Let's specify the m, b, k values and plot Displacement vs. Time graph.

In [0]:
m = 10     #[kg]
b = 50     #[N/m/s]
k = 1000   #[N/m]
num = [1]
den = [m,b,k]
TF = cnt.tf(num,den)
print("G(s)=",TF)
#Displacement vs. Time Plot
t = np.linspace(0,5,501)   #time duration between 0 and 5 s divided into 501 points
t,x = cnt.step_response(TF,t)   #step response of the system from transfer function where t is the time and x is the displacement of mass
plt.plot(t,x)
plt.xlabel("Time [s]")
plt.ylabel("Displacement of mass")
plt.show()

Change the variables(m, b, k), observe the transfer function and see the difference in the displacement of mass accordingly.

In [0]:
m = ???     #[kg]
b = ???     #[N/m/s]
k = ???   #[N/m]
num = [1]
den = [m,b,k]
TF = cnt.tf(num,den)
print("G(s)=",TF)
#Displacement vs. Time Plot
t = np.linspace(0,5,501)   #time duration between 0 and 5 s divided into 501 points
t,x = cnt.step_response(TF,t)   #step response of the system from transfer function where t is the time and x is the displacement of mass
plt.plot(t,x)
plt.xlabel("Time [s]")
plt.ylabel("Displacement of mass [m]")
plt.show()



---



---


Example 2 - Second Order System


---



---






<figure>
<center>
<img src='https://www.howacarworks.com/illustration/127/coil-spring.png' />
<figcaption>Figure 2</figcaption></center>



The suspension system of a vehicle is shown in *Figure 2* and a simple dynamic model of a vehicle travelling on a rough road surface is shown in *Figure 3*. The mass represents the mass of the vehicle body. The spring and damper represent the suspension springs and dampers.

<figure>
<center>
<img src='https://live.staticflickr.com/65535/49904889778_9580484706_b.jpg' />
<figcaption>Figure 3</figcaption></center>
</figure>

The input is the road profile displacement, ie. $z $. If the output is the displacement of the body ie. y. 


*   Obtain input-output relationship as $m\ddot{y}+c\dot{y}+ky=c\dot{z}+kz$ .
*   In order to get transfer function, we should get the laplace transform of input-output expression.
$ms^2Y(s)+csY(s)+kY(s)=csZ(s)+kZ(s)$
then 
$\dfrac {output} {input} = \dfrac {Y(s)} {Z(s)} = \dfrac {cs+k} {ms^2+cs+k}  $.



Let's specify the m, c, k values and plot Displacement vs. Time graph to understand transfer function's effects.

In [0]:
m = 1600     #[kg]
c = 6000     #[N/m/s]
k = 784000   #[N/m]
num = [c,k]
den = [m,c,k]
TF = cnt.tf(num,den)
print("G(s)=",TF)
#Displacement vs. Time Plot
t = np.linspace(0,5,501)   #time duration between 0 and 5 s divided into 501 points
t,y = cnt.step_response(TF,t)   #step response of the system from transfer function where t is the time and y is the displacement of car
plt.plot(t,y)
plt.xlabel("Time [s]")
plt.ylabel("Displacement of car body in y-axis")
plt.show()

Now, it's your turn. You can change the m, c, k values and see the difference in the displacement, which depends on the transfer function.

In [0]:
m =  ???    #[kg]
c =  ???    #[N/m/s]
k =  ???    #[N/m]
num = [c,k]
den = [m,c,k]
TF = cnt.tf(num,den)
print("G(s)=",TF)
#Displacement vs. Time Plot
t = np.linspace(0,5,501)   #time duration between 0 and 5 s divided into 501 points
t,y = cnt.step_response(TF,t)   #step response of the system from transfer function where t is the time and y is the displacement of car
plt.plot(t,y)
plt.xlabel("Time [s]")
plt.ylabel("displacement of car body in y-axis")
plt.show()